# Worksheet 08

Name: Stone Harris
UID: U41533031

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [6]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.916172589697032, 3.748718004609172, 5.382649434265281, 4.1508940731772315, 4.326618748319526, 6.003143282990388, 4.414942852507401, 4.893711829189769, 7.281500053866833, 6.023034787521693]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [8]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.748796586569009, 8.178678618279521, 7.095859433933889, 9.672936569349467, 7.765196373426297, 7.1733654735956085, 6.476108043406354, 8.607702432538352, 7.606694162594711, 6.688665067765635]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [9]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.389550625755689, 3.9056998543985344, 6.688665067765635, 7.234326621661057, 7.606694162594711, 4.759306030616764, 8.607702432538352, 5.924999071521329, 6.476108043406354, 7.1733654735956085]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Means (μ1, μ2): Centers of the Gaussian distributions. Each mixture component has a vector, which indicates where the center of the cluster is located.

- Covariances (Σ1, Σ2): Represent how the data points are spread out around the mean. Covariance matrices define the shape and orientation of the data as it is distributed around the mean.

- Mixture Weights (π1, π2): These are the weights of each Gaussian component, indicating the probability that a randomly selected data point belongs to a particular component.

- Number of Components (K): In this case, K=2 as specified by having two mixture components.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [10]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]

mean_1 = sum(s1)/len(s1)
mean_2 = sum(s2)/len(s2)
mean = [mean_1, mean_2]

var_1 = sum((x - mean_1)**2 for x in s1) / len(s1)
var_2 = sum((x - mean_2)**2 for x in s2) / len(s2)
var = [var_1, var_2]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.389550625755689, 3.9056998543985344, 4.759306030616764, 5.924999071521329, 6.476108043406354]
[6.688665067765635, 7.234326621661057, 7.606694162594711, 8.607702432538352, 7.1733654735956085]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.4911327251397335,  mean_2 = 7.462150751631073
var_1 = 1.0029477613414444,  var_2 = 0.41335246508677875


I think that the estimates are relatively close to the true values. I'm not sure what exactly we are considering 'close', but I think that they are close enough.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [11]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.389550625755689
probability of observing that point if it came from cluster 0 =  0.2663109946756367
probability of observing that point if it came from cluster 1 =  0.03330041142479493
point =  3.9056998543985344
probability of observing that point if it came from cluster 0 =  0.11402884904314586
probability of observing that point if it came from cluster 1 =  8.123461826556302e-17
point =  6.688665067765635
probability of observing that point if it came from cluster 0 =  0.19500792186747332
probability of observing that point if it came from cluster 1 =  0.16758381517570173
point =  7.234326621661057
probability of observing that point if it came from cluster 0 =  0.08783225696923823
probability of observing that point if it came from cluster 1 =  0.8291338896140591
point =  7.606694162594711
probability of observing that point if it came from cluster 0 =  0.04300034408696409
probability of observing that point if it came from cluster 1 =  0.9078973801262846
point =  4.7593

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [12]:
prob_c = [sum(prob_s0_x) / len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x)]
mean = [sum(p * x for p, x in zip(prob_s0_x, data)) / sum(prob_s0_x),
        sum(p * x for p, x in zip(prob_s1_x, data)) / sum(prob_s1_x)]
var = [sum(p * (x - mean[0])**2 for p, x in zip(prob_s0_x, data)) / sum(prob_s0_x),
       sum(p * (x - mean[1])**2 for p, x in zip(prob_s1_x, data)) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.709619635434563,  mean_2 = 7.463339739403822
var_1 = 1.307384982710878,  var_2 = 0.4275183624499728


These values are very similar to the ones that I obtained from using K means. This largest difference that I have observed in results is only 0.3. 

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [13]:
from scipy.stats import norm
prob_s0_x = [] 
prob_s1_x = []  

for p in data:
    pdf_s0 = norm.pdf(p, mean[0], np.sqrt(var[0]))
    pdf_s1 = norm.pdf(p, mean[1], np.sqrt(var[1]))
    prob_x_i = prob_c[0] * pdf_s0 + prob_c[1] * pdf_s1

    prob_s0_x_i = (pdf_s0 * prob_c[0]) / prob_x_i
    prob_s1_x_i = (pdf_s1 * prob_c[1]) / prob_x_i

    prob_s0_x.append(prob_s0_x_i)
    prob_s1_x.append(prob_s1_x_i)

for i, p in enumerate(data):
    print(f"Data point: {p}")
    print(f"P(S_1 | X_{i}) updated: {prob_s0_x[i]}")
    print(f"P(S_2 | X_{i}) updated: {prob_s1_x[i]}\n")

Data point: 6.389550625755689
P(S_1 | X_0) updated: 0.7036325694601052
P(S_2 | X_0) updated: 0.2963674305398947

Data point: 3.9056998543985344
P(S_1 | X_1) updated: 0.99999824155413
P(S_2 | X_1) updated: 1.7584458699896062e-06

Data point: 6.688665067765635
P(S_1 | X_2) updated: 0.5070589271715339
P(S_2 | X_2) updated: 0.49294107282846605

Data point: 7.234326621661057
P(S_1 | X_3) updated: 0.2432794901629256
P(S_2 | X_3) updated: 0.7567205098370743

Data point: 7.606694162594711
P(S_1 | X_4) updated: 0.15984513730117716
P(S_2 | X_4) updated: 0.8401548626988228

Data point: 4.759306030616764
P(S_1 | X_5) updated: 0.9996290291048616
P(S_2 | X_5) updated: 0.0003709708951384327

Data point: 8.607702432538352
P(S_1 | X_6) updated: 0.1205165543660526
P(S_2 | X_6) updated: 0.8794834456339474

Data point: 5.924999071521329
P(S_1 | X_7) updated: 0.9200423118737572
P(S_2 | X_7) updated: 0.07995768812624288

Data point: 6.476108043406354
P(S_1 | X_8) updated: 0.6475148007131555
P(S_2 | X_8) upd

After updating the probabilities with P(S_j | X_i), there seems to be a clearer distinction between which cluster a point is more likely to belong to.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [19]:
hard_assignments = []

for prob_s0, prob_s1 in zip(prob_s0_x, prob_s1_x):
    if prob_s0 > prob_s1:
        hard_assignments.append(0) 
    else:
        hard_assignments.append(1)  

for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} is assigned to cluster {assignment}")

Data point 0 is assigned to cluster 0
Data point 1 is assigned to cluster 0
Data point 2 is assigned to cluster 0
Data point 3 is assigned to cluster 1
Data point 4 is assigned to cluster 1
Data point 5 is assigned to cluster 0
Data point 6 is assigned to cluster 1
Data point 7 is assigned to cluster 0
Data point 8 is assigned to cluster 0
Data point 9 is assigned to cluster 1
